In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import regex as re

Fil = pd.read_pickle('../data/raw/Fil.pkl')
Dokument = pd.read_pickle('../data/raw/Dokument.pkl')
Sag = pd.read_pickle('../data/raw/Sag.pkl')
SagDokument = pd.read_pickle('../data/raw/SagDokument.pkl')

This notebook identifies the Files that are not relevant for the model
These are files that are duplicates in some way. Either by :
1. Being the question for which the answer file contains the question
2. Being an intermediate file, where a final answer exists
3. Being an EU file, where a subsequent national file exists

The notebook then extracts a random subset in order to fine tuen a BERT emebedding model

In [ ]:
Dokument['kategoriid'].value_counts()[0:20]
#man kan nok godt fjerne udvalgsspørsmål (36), samrådsspørgsmål (37),  foreløbigt svar (30) og  §20 spørgsmål (14)
#det sparer 245123 +23212+ 53136 + 15763 = 335834 = ca. 40% af dokumenterne
#man kan også fjerne alle dokumenter relateret til sager med typeid 7, da disse er EU sager

In [ ]:
len(Dokument)

In [ ]:
unwanted_categories = [36,37,30,14]

In [ ]:
Not_EU_Sag = Sag[Sag['typeid'] != 7]
Dokument_ids = SagDokument[SagDokument['sagid'].isin(Not_EU_Sag['id'])]['dokumentid'].unique()
len(Dokument_ids)

In [ ]:
Filtered_dok = Dokument[~Dokument['kategoriid'].isin(unwanted_categories)]
len(Filtered_dok)

In [ ]:
Filtered_dok = Filtered_dok[Filtered_dok['id'].isin(Dokument_ids)]
len(Filtered_dok)

In [ ]:
Filtered_fil = Fil[Fil['dokumentid'].isin(Filtered_dok['id'])]
len(Filtered_fil)

In [ ]:
#pickle the filtered dataframes
Filtered_dok.to_pickle('../data/raw/Filtered_dok.pkl')
Filtered_fil.to_pickle('../data/raw/Filtered_fil.pkl')

In [ ]:
#taking dokument have a spørgsmålsid that is not none and a kategoriid == 22
question_doks = Filtered_dok[~Filtered_dok['spørgsmålsid'].isna() & (Filtered_dok['kategoriid'] == 22)]['titel'].tolist()
question_ids = Filtered_dok[~Filtered_dok['spørgsmålsid'].isna() & (Filtered_dok['kategoriid'] == 22)]['id'].tolist()

In [ ]:
filtered_questions = []
import re 
pattern = r'^((Svar på spm\. nr\. S \d+:)|(\w+ - svar på spm\. \d+)|Svar på KOM \(\d+\) \d+ spm\. \d+,?\s+om\s+|(Aktstk\.? \d+ - svar på spm\. \d+)|([A-Z]{3} \(\d+\) \d+ - svar på spm\. \d+)|(svar på spm\. nr\. S \d+:)|(L \d+ - svar på spm\. \d+)|(Svar på spm\.nr\. S \d+:)|([A-Z]{3} alm\. del - svar på spm\. \d+)|(UUI alm\. del - svar på spm\. \d+)|(Svar på spm\. nr\. \d+:)|([A-Z]{3} alm\. del - svar på spm\. \d+)|(URU alm\. del - svar på spm\. \d+)|(B \d+ - svar på spm\. \d+)|(KEB alm\. del - svar på MFU spm\. \d+)|(TRU alm\. del - svar på spm\. \d+))'

for question in question_doks:
    filtered_question = re.sub(pattern, '', question)
    filtered_questions.append(filtered_question)



In [ ]:
zip_iterator = zip(question_ids, filtered_questions)
question_dict = dict(zip_iterator)

In [ ]:
import random
random.seed(42)
random_subset = random.sample(list(question_dict.items()), 10000)

In [ ]:
random_subset
import pickle
with open('../data/raw/random_subset.pkl', 'wb') as f:
    pickle.dump(random_subset, f)